In [ ]:
from utils.tensorrt import InferenceEngine

In [ ]:
from pycuda import autoinit

In [ ]:
model = InferenceEngine(
    "models/retinanet_540p.onnx",
    "models/retinanet_540p.engine",
    fp16_mode=True,
)